In [1]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
if 0:
    print('Load street graph')
    G = snman.io.load_street_graph(
        os.path.join(outputs_path, 'G_edges.gzip'),
        os.path.join(outputs_path, 'G_nodes.gzip'),
        crs=CRS_internal
    )
    
    snman.street_graph.organize_edge_directions(G)
    
    nodes, edges = oxc.graph_to_gdfs(G)

In [3]:
# Import Statent grid
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)
statent

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
0,2021,2023,2632200,1247300,63222473,4,0,4,4,7,...,0.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,0.0,POINT (2632200.000 1247300.000)
1,2021,2023,2632300,1248100,63232481,4,4,0,0,4,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2632300.000 1248100.000)
2,2021,2023,2632400,1247300,63242473,4,0,0,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632400.000 1247300.000)
3,2021,2023,2632400,1247500,63242475,4,0,4,0,4,...,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,POINT (2632400.000 1247500.000)
4,2021,2023,2632500,1247200,63252472,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632500.000 1247200.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58739,2021,2023,2728300,1222500,72832225,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728300.000 1222500.000)
58740,2021,2023,2728300,1222900,72832229,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,POINT (2728300.000 1222900.000)
58741,2021,2023,2728400,1222700,72842227,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728400.000 1222700.000)
58742,2021,2023,2728500,1222500,72852225,4,4,0,0,4,...,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2728500.000 1222500.000)


In [4]:
# read statpop that has been pre-joined with statent
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
)

statpop.head()

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
12,2017,1962,1,2,302,55,3,896883164,POINT (2674380.001 1250094.001),18914.0,67442501.0,20.879700
15,2017,1966,2,1,70102,51,7,899882330,POINT (2672299.001 1251208.001),17671.0,67232512.0,8.063317
16,2017,1984,1,1,13,33,13,899882659,POINT (2673496.001 1250355.001),NaN,NaN,NaN
20,2017,1992,1,2,201,25,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383
21,2017,1994,2,2,201,23,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383


In [5]:
# filter statpop for one single statent cell
statent_id = 68252512
statpop_filtered = statpop.query(f'statent_id == {statent_id}')
statpop_filtered.head()

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
234507,2017,1981,2,2,-2,36,-2,597939907,POINT (2682548.001 1251183.001),28059.0,68252512.0,50.921708
274900,2017,1972,2,2,-2,45,-2,899849080,POINT (2682502.001 1251169.001),28059.0,68252512.0,31.063351
379693,2017,2007,1,1,-2,10,-2,597939911,POINT (2682545.001 1251248.001),28059.0,68252512.0,65.796390
417369,2017,2010,1,1,-2,7,-2,597939906,POINT (2682532.001 1251207.001),28059.0,68252512.0,32.757524
637853,2017,1929,2,3,-2,88,-2,597939911,POINT (2682545.001 1251248.001),28059.0,68252512.0,65.796390


In [6]:
# filter statent for cells within distance limit
origin = statent.set_index('id').geometry[statent_id]
origins = geopandas.GeoDataFrame(
    {
        "id": [statent_id],
        "geometry": [origin]
    },
    crs="EPSG:2056",
)
statent_filtered = statent[statent['geometry'].within(
    origin.buffer(10000)
)]
statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
17910,2021,2023,2672600,1249900,67262499,4,0,4,4,4,...,0.0,0.0,0.0,4.000000,0.000000,4.000000,4.000000,0.000000,4.000000,POINT (2672600.000 1249900.000)
17911,2021,2023,2672600,1250000,67262500,4,0,4,4,4,...,0.0,0.0,0.0,4.000000,0.000000,4.000000,4.000000,4.000000,4.000000,POINT (2672600.000 1250000.000)
17912,2021,2023,2672600,1250200,67262502,4,0,4,4,25,...,0.0,0.0,0.0,4.000000,4.000000,4.000000,12.088934,7.770000,4.318934,POINT (2672600.000 1250200.000)
17913,2021,2023,2672600,1250300,67262503,4,0,0,4,7,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.651834,4.000000,4.000000,POINT (2672600.000 1250300.000)
17914,2021,2023,2672600,1250400,67262504,6,0,0,6,225,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,159.777359,133.092159,26.685200,POINT (2672600.000 1250400.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39770,2021,2023,2692300,1249400,69232494,4,0,0,4,55,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,41.370000,33.580000,7.790000,POINT (2692300.000 1249400.000)
39771,2021,2023,2692300,1249600,69232496,4,0,4,4,26,...,0.0,0.0,0.0,22.000000,0.000000,22.000000,4.000000,4.000000,4.000000,POINT (2692300.000 1249600.000)
39772,2021,2023,2692300,1249700,69232497,10,0,6,4,117,...,0.0,0.0,0.0,101.916752,5.680991,96.235761,11.159806,4.000000,9.586293,POINT (2692300.000 1249700.000)
39773,2021,2023,2692300,1249800,69232498,4,0,0,4,4,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,POINT (2692300.000 1249800.000)


In [7]:
transport_network = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
    #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip')
)

In [8]:
print('read transport network cycling')
transport_network_cycling = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_cycling.osm.pbf'),
)
print('read transport network car')
transport_network_car = r5py.TransportNetwork(
    os.path.join(outputs_path, 'tt.osm.pbf'),
)

read transport network cycling
read transport network car


let R5 calculate the travel time matrix between all statent cells

In [24]:
tt_computers = {}
common_settings = {
    'origins': origins,
    'destinations': statent_filtered,
    'departure': datetime.datetime(2024, 2, 22, 8, 30),
}

tt_computers[MODE_FOOT] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.WALK],
    **common_settings
)

tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network_cycling,
    transport_modes=[r5py.TransportMode.CAR],
    **common_settings
)

tt_computers[MODE_PRIVATE_CARS] = r5py.TravelTimeMatrixComputer(
    transport_network_car,
    transport_modes=[r5py.TransportMode.CAR],
    **common_settings
)

tt_computers[MODE_TRANSIT] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.TRANSIT],
    **common_settings
)

# travel time matrix calculation: compute travel times, add a mode label
def tt_calculation(tt_computer, mode):
    tt_matrix = tt_computer.compute_travel_times()
    tt_matrix['mode'] = mode
    return tt_matrix

# calculate a travel time matrix for each mode
tt_matrices = {mode: tt_calculation(tt_computer, mode) for mode, tt_computer in tt_computers.items()}

# merge all travel time matrices into one
tt_matrix = pd.concat(tt_matrices.values())

# add base travel time to avoid 0 values
tt_matrix['travel_time'] = tt_matrix['travel_time'] + 0.1
tt_matrix[tt_matrix['mode'] == 'foot']

C:\Users\lballo\AppData\Local\miniconda3\envs\snman\lib\site-packages\r5py\r5\regional_task.py:223: RuntimeWarning: Departure time 2024-02-22 08:30:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(


,from_id,to_id,travel_time,mode
0,68252512,67262499,NaN,foot
1,68252512,67262500,NaN,foot
2,68252512,67262502,NaN,foot
3,68252512,67262503,NaN,foot
4,68252512,67262504,NaN,foot
...,...,...,...,...
10004,68252512,69232494,NaN,foot
10005,68252512,69232496,NaN,foot
10006,68252512,69232497,NaN,foot
10007,68252512,69232498,NaN,foot


prepare a reduced form of statpop

In [25]:
statpop_reduced = statpop_filtered.reset_index()[[
    'record', 'age', 'maritalstatus', 'residencepermit', 'residentpermit', 'statent_id', 'geometry'
]]
statpop_reduced.set_index('record', inplace=True)
statpop_reduced.head()

,age,maritalstatus,residencepermit,residentpermit,statent_id,geometry
record,,,,,,
234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001)
274900,45,2,-2,-2,68252512.0,POINT (2682502.001 1251169.001)
379693,10,1,-2,-2,68252512.0,POINT (2682545.001 1251248.001)
417369,7,1,-2,-2,68252512.0,POINT (2682532.001 1251207.001)
637853,88,3,-2,-2,68252512.0,POINT (2682545.001 1251248.001)


merge statent and statpop: get a list of persons with their travel time to each destination

In [26]:
accessibility_costs = pd.merge(statpop_reduced.reset_index(), tt_matrix, how='left', left_on='statent_id', right_on='from_id')
#accessibility_costs = accessibility_costs[accessibility_costs['travel_time'].notna()]
accessibility_costs

,record,age,maritalstatus,residencepermit,residentpermit,statent_id,geometry,from_id,to_id,travel_time,mode
0,234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001),68252512,67262499,NaN,foot
1,234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001),68252512,67262500,NaN,foot
2,234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001),68252512,67262502,NaN,foot
3,234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001),68252512,67262503,NaN,foot
4,234507,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001),68252512,67262504,NaN,foot
...,...,...,...,...,...,...,...,...,...,...,...
5765179,12723489,48,2,302,3,68252512.0,POINT (2682455.001 1251225.001),68252512,69232494,NaN,transit
5765180,12723489,48,2,302,3,68252512.0,POINT (2682455.001 1251225.001),68252512,69232496,NaN,transit
5765181,12723489,48,2,302,3,68252512.0,POINT (2682455.001 1251225.001),68252512,69232497,NaN,transit
5765182,12723489,48,2,302,3,68252512.0,POINT (2682455.001 1251225.001),68252512,69232498,NaN,transit


In [27]:
# group accessibility by person, providing a list of travel times to each statent destination

ag = accessibility_costs.reset_index().groupby(['record', 'to_id']).agg({
    'travel_time': list,
    'mode': list,
    'age': 'first',
    'from_id': 'first'
})

# add destination information from statent
ag = pd.merge(ag.reset_index(), statent.reset_index()[['id', 'VOLLZEITAEQ_TOTAL']], left_on='to_id', right_on='id')

ag

,record,to_id,travel_time,mode,age,from_id,id,VOLLZEITAEQ_TOTAL
0,234507,67262499,"[nan, 42.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262499,4.000000
1,234507,67262500,"[nan, 42.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262500,4.000000
2,234507,67262502,"[nan, 41.1, 17.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262502,15.236510
3,234507,67262503,"[nan, 38.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262503,4.651834
4,234507,67262504,"[nan, 37.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262504,159.777359
...,...,...,...,...,...,...,...,...
1441291,12723489,69232494,"[nan, 38.1, 19.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232494,41.370000
1441292,12723489,69232496,"[nan, 38.1, 19.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232496,24.422705
1441293,12723489,69232497,"[nan, 39.1, 17.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232497,113.076558
1441294,12723489,69232498,"[nan, 40.1, 17.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232498,4.000000


In [28]:
ag['travel_options'] = ag.apply(
    lambda row: dict(
        zip(
            row['mode'], row['travel_time']
        )
    ),
    axis=1
)

In [29]:
ag

,record,to_id,travel_time,mode,age,from_id,id,VOLLZEITAEQ_TOTAL,travel_options
0,234507,67262499,"[nan, 42.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262499,4.000000,"{'foot': nan, 'cycling': 42.1, 'private_cars':..."
1,234507,67262500,"[nan, 42.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262500,4.000000,"{'foot': nan, 'cycling': 42.1, 'private_cars':..."
2,234507,67262502,"[nan, 41.1, 17.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262502,15.236510,"{'foot': nan, 'cycling': 41.1, 'private_cars':..."
3,234507,67262503,"[nan, 38.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262503,4.651834,"{'foot': nan, 'cycling': 38.1, 'private_cars':..."
4,234507,67262504,"[nan, 37.1, 16.1, nan]","[foot, cycling, private_cars, transit]",36,68252512,67262504,159.777359,"{'foot': nan, 'cycling': 37.1, 'private_cars':..."
...,...,...,...,...,...,...,...,...,...
1441291,12723489,69232494,"[nan, 38.1, 19.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232494,41.370000,"{'foot': nan, 'cycling': 38.1, 'private_cars':..."
1441292,12723489,69232496,"[nan, 38.1, 19.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232496,24.422705,"{'foot': nan, 'cycling': 38.1, 'private_cars':..."
1441293,12723489,69232497,"[nan, 39.1, 17.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232497,113.076558,"{'foot': nan, 'cycling': 39.1, 'private_cars':..."
1441294,12723489,69232498,"[nan, 40.1, 17.1, nan]","[foot, cycling, private_cars, transit]",48,68252512,69232498,4.000000,"{'foot': nan, 'cycling': 40.1, 'private_cars':..."


In [14]:
# run the mode choice model to calculate each person's generalized cost to each statent destination

def calculate_behavioral_cost(private_cars=9999999, transit=9999999, cycling=9999999, foot=9999999, age=None):
    """
    Calculates individual generalized cost based on the personal properties and mode choice.
    Based on mode choice model in
    
    Hörl, S., M. Balać and K.W. Axhausen (2019)
    Pairing discrete mode choice models and agent-based transport simulation with MATSim,
    paper presented at the 98th Annual Meeting of the Transportation Research Board (TRB 2019),
    Washington DC, January 2019. 
    
    Returns a tuple of mode specific choice probabilities (dict) and the resulting behavioral travel time (float)

    Parameters
    ----------
    private_cars: float
        travel time in minutes
    transit: float
        travel time in minutes
    cycling: float
        travel time in minutes
    foot: float
        travel time in minutes
    age: int
        age in years

    Returns
    -------
    (dict, float)
    """
    
    # mode-specific costs (travel times in minutes)
    tt = {
        MODE_PRIVATE_CARS: private_cars,
        MODE_TRANSIT: transit,
        MODE_CYCLING: cycling,
        MODE_FOOT: foot
    }
    
    # mode choice model parameters from Hörl et al. (2019), pp. 11, Table 1
    # t: time
    
    alpha_car =                    0.827   # [-]
    beta_tt_car =                 -0.0667  # [min-1]
    
    alpha_pt =                     0.0     # [-]
    beta_transfer =               -0.17    # [-]
    beta_in_vehicle_t =           -0.0192  # [min-1]
    beta_transfer_t =             -0.0384  # [min-1]
    beta_access_egress_t =        -0.0804  # [min-1]
    
    alpha_bike =                  -0.1     # [-]
    beta_tt_bike =                -0.0805  # [min-1]
    beta_age_bike =               -0.0496  # [years-1]
    
    alpha_walk =                   0.63    # [-]
    beta_tt_walk =                -0.141   # [min-1]
    
    beta_cost =                   -0.126   # [CHF-1]
    lambda_factor =               -0.4     # [-1]
    phi_avg_distance =            40.0     # [km]
    
    phi_parking_search_penalty =   6.0     # [min]
    phi_access_egress_walk_t =     5.0     # [min]
    
    # surrogate parameters to replace missing information (R5 provides only travel time)
    avg_speed_car =               25.0     # [kmh]
    detour_parameter_car =         1.3     # [-]
    x_crowfly_distance = tt[MODE_PRIVATE_CARS] / 60 * avg_speed_car / detour_parameter_car
    x_cost_car = tt[MODE_PRIVATE_CARS] * 0.2
    x_cost_pt = tt[MODE_TRANSIT] * 0.2
    
    x_transfers = round(tt[MODE_TRANSIT] / 30) if tt[MODE_TRANSIT] is not np.inf else np.inf
    x_transfer_t = x_transfers * 5
    x_access_egress_t = 5
    
    # mode-specific utilities from Hörl et al. (2019), pp. 10, eq. 3-6
    u = {
        MODE_PRIVATE_CARS: 
            alpha_car
            + beta_tt_car * tt[MODE_PRIVATE_CARS]
            + beta_tt_car * phi_parking_search_penalty
            + beta_tt_walk * phi_access_egress_walk_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_car
        ,
        MODE_TRANSIT:
            alpha_pt
            + beta_transfer * x_transfers
            + beta_transfer_t * x_transfer_t
            + beta_access_egress_t * x_access_egress_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_pt
        ,
        MODE_CYCLING:      
            alpha_bike
            + beta_tt_bike * tt[MODE_CYCLING]
            + beta_age_bike * max(0, age - 18)
        ,
        MODE_FOOT:
            alpha_walk
            + beta_tt_walk * tt[MODE_FOOT]
    }

    # mode-specific choice probabilities
    denominator = sum([np.exp(u_mode) for u_mode in u.values()])
    P = {mode: np.exp(u_mode) / denominator for mode, u_mode in u.items()}
    
    # total cost, after weighting the mode-specific cost by mode-specific choice probabilities
    tt_total = sum([tt[mode] * P[mode] for mode in tt.keys()])
    return P, tt_total

ag[['choice_probabilities', 'behavioral_cost']] = ag.apply(
    lambda row: calculate_behavioral_cost(**row['travel_options'], age=row['age']),
    axis=1,
    result_type='expand'
)

ag = pd.concat([
    ag,
    pd.json_normalize(ag['choice_probabilities']).add_prefix('prob_'),
    pd.json_normalize(ag['travel_options']).add_prefix('tt_')
    ],
    axis=1
)

ag['accessibility_contribution'] = ag['behavioral_cost'] ** -0.7

ag

,record,to_id,travel_time,mode,age,from_id,id,VOLLZEITAEQ_TOTAL,travel_options,choice_probabilities,behavioral_cost,prob_private_cars,prob_transit,prob_cycling,prob_foot,tt_cycling,tt_private_cars,tt_foot,tt_transit,accessibility_contribution
0,234507,67262499,"[42.1, 16.1]","[cycling, private_cars]",36,68252512,67262499,4.000000,"{'cycling': 42.1, 'private_cars': 16.1}","{'private_cars': 0.8918104460471433, 'transit'...",18.912928,0.891810,0.0,0.108190,0.0,42.1,16.1,NaN,NaN,0.127723
1,234507,67262500,"[42.1, 16.1]","[cycling, private_cars]",36,68252512,67262500,4.000000,"{'cycling': 42.1, 'private_cars': 16.1}","{'private_cars': 0.8918104460471433, 'transit'...",18.912928,0.891810,0.0,0.108190,0.0,42.1,16.1,NaN,NaN,0.127723
2,234507,67262502,"[41.1, 17.1]","[cycling, private_cars]",36,68252512,67262502,15.236510,"{'cycling': 41.1, 'private_cars': 17.1}","{'private_cars': 0.8730591346020291, 'transit'...",20.146581,0.873059,0.0,0.126941,0.0,41.1,17.1,NaN,NaN,0.122197
3,234507,67262503,"[38.1, 16.1]","[cycling, private_cars]",36,68252512,67262503,4.651834,"{'cycling': 38.1, 'private_cars': 16.1}","{'private_cars': 0.8566043791460333, 'transit'...",19.254704,0.856604,0.0,0.143396,0.0,38.1,16.1,NaN,NaN,0.126132
4,234507,67262504,"[37.1, 16.1]","[cycling, private_cars]",36,68252512,67262504,159.777359,"{'cycling': 37.1, 'private_cars': 16.1}","{'private_cars': 0.8464297104054326, 'transit'...",19.324976,0.846430,0.0,0.153570,0.0,37.1,16.1,NaN,NaN,0.125810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441291,12723489,69232494,"[38.1, 19.1]","[cycling, private_cars]",48,68252512,69232494,41.370000,"{'cycling': 38.1, 'private_cars': 19.1}","{'private_cars': 0.8892497263352831, 'transit'...",21.204255,0.889250,0.0,0.110750,0.0,38.1,19.1,NaN,NaN,0.117897
1441292,12723489,69232496,"[38.1, 19.1]","[cycling, private_cars]",48,68252512,69232496,24.422705,"{'cycling': 38.1, 'private_cars': 19.1}","{'private_cars': 0.8892497263352831, 'transit'...",21.204255,0.889250,0.0,0.110750,0.0,38.1,19.1,NaN,NaN,0.117897
1441293,12723489,69232497,"[39.1, 17.1]","[cycling, private_cars]",48,68252512,69232497,113.076558,"{'cycling': 39.1, 'private_cars': 17.1}","{'private_cars': 0.9139212583759464, 'transit'...",18.993732,0.913921,0.0,0.086079,0.0,39.1,17.1,NaN,NaN,0.127342
1441294,12723489,69232498,"[40.1, 17.1]","[cycling, private_cars]",48,68252512,69232498,4.000000,"{'cycling': 40.1, 'private_cars': 17.1}","{'private_cars': 0.9200467164881905, 'transit'...",18.938926,0.920047,0.0,0.079953,0.0,40.1,17.1,NaN,NaN,0.127600


In [19]:
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    ag[f'accessibility_contribution_{mode}'] = ag[f'tt_{mode}'] ** -0.7
    
ag

,record,to_id,travel_time,mode,age,from_id,id,VOLLZEITAEQ_TOTAL,travel_options,choice_probabilities,...,prob_foot,tt_cycling,tt_private_cars,tt_foot,tt_transit,accessibility_contribution,accessibility_contribution_cycling,accessibility_contribution_private_cars,accessibility_contribution_transit,accessibility_contribution_foot
0,234507,67262499,"[42.1, 16.1]","[cycling, private_cars]",36,68252512,67262499,4.000000,"{'cycling': 42.1, 'private_cars': 16.1}","{'private_cars': 0.8918104460471433, 'transit'...",...,0.0,42.1,16.1,NaN,NaN,0.127723,0.072946,0.142962,NaN,NaN
1,234507,67262500,"[42.1, 16.1]","[cycling, private_cars]",36,68252512,67262500,4.000000,"{'cycling': 42.1, 'private_cars': 16.1}","{'private_cars': 0.8918104460471433, 'transit'...",...,0.0,42.1,16.1,NaN,NaN,0.127723,0.072946,0.142962,NaN,NaN
2,234507,67262502,"[41.1, 17.1]","[cycling, private_cars]",36,68252512,67262502,15.236510,"{'cycling': 41.1, 'private_cars': 17.1}","{'private_cars': 0.8730591346020291, 'transit'...",...,0.0,41.1,17.1,NaN,NaN,0.122197,0.074184,0.137057,NaN,NaN
3,234507,67262503,"[38.1, 16.1]","[cycling, private_cars]",36,68252512,67262503,4.651834,"{'cycling': 38.1, 'private_cars': 16.1}","{'private_cars': 0.8566043791460333, 'transit'...",...,0.0,38.1,16.1,NaN,NaN,0.126132,0.078226,0.142962,NaN,NaN
4,234507,67262504,"[37.1, 16.1]","[cycling, private_cars]",36,68252512,67262504,159.777359,"{'cycling': 37.1, 'private_cars': 16.1}","{'private_cars': 0.8464297104054326, 'transit'...",...,0.0,37.1,16.1,NaN,NaN,0.125810,0.079696,0.142962,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441291,12723489,69232494,"[38.1, 19.1]","[cycling, private_cars]",48,68252512,69232494,41.370000,"{'cycling': 38.1, 'private_cars': 19.1}","{'private_cars': 0.8892497263352831, 'transit'...",...,0.0,38.1,19.1,NaN,NaN,0.117897,0.078226,0.126846,NaN,NaN
1441292,12723489,69232496,"[38.1, 19.1]","[cycling, private_cars]",48,68252512,69232496,24.422705,"{'cycling': 38.1, 'private_cars': 19.1}","{'private_cars': 0.8892497263352831, 'transit'...",...,0.0,38.1,19.1,NaN,NaN,0.117897,0.078226,0.126846,NaN,NaN
1441293,12723489,69232497,"[39.1, 17.1]","[cycling, private_cars]",48,68252512,69232497,113.076558,"{'cycling': 39.1, 'private_cars': 17.1}","{'private_cars': 0.9139212583759464, 'transit'...",...,0.0,39.1,17.1,NaN,NaN,0.127342,0.076820,0.137057,NaN,NaN
1441294,12723489,69232498,"[40.1, 17.1]","[cycling, private_cars]",48,68252512,69232498,4.000000,"{'cycling': 40.1, 'private_cars': 17.1}","{'private_cars': 0.9200467164881905, 'transit'...",...,0.0,40.1,17.1,NaN,NaN,0.127600,0.075474,0.137057,NaN,NaN


In [16]:
import shapely as shp

ag2 = pd.merge(
    ag,
    statpop_reduced[['geometry']],
    left_on='record', right_index=True
).rename(columns={'geometry': 'home_geometry'})

ag2 = pd.merge(
    ag2,
    statent_filtered[['id', 'geometry']],
    left_on='from_id', right_on='id'
).rename(columns={'geometry': 'origin_geometry'})

ag2 = pd.merge(
    ag2,
    statent_filtered[['id', 'geometry']],
    left_on='to_id', right_on='id'
).rename(columns={'geometry': 'destination_geometry'})

ag2['geometry'] = ag2.apply(
    lambda row: shp.LineString([
        row['home_geometry'],
        row['origin_geometry'],
        row['destination_geometry']
    ]),
    axis=1
)

ag2.drop(
    columns=[
        'home_geometry', 'origin_geometry', 'destination_geometry',
        'id', 'id_x', 'id_y',
        'travel_time', 'mode',
        'travel_options', 'choice_probabilities'
    ],
    inplace=True
)

ag2 = gpd.GeoDataFrame(ag2, crs=2056)

ag2

,record,to_id,age,from_id,VOLLZEITAEQ_TOTAL,behavioral_cost,prob_private_cars,prob_transit,prob_cycling,prob_foot,tt_cycling,tt_private_cars,tt_foot,tt_transit,accessibility_contribution,geometry
0,234507,67262499,36,68252512,4.000000,18.912928,0.891810,0.0,0.108190,0.0,42.1,16.1,NaN,NaN,0.127723,"LINESTRING (2682548.001 1251183.001, 2682500.0..."
1,234507,67262500,36,68252512,4.000000,18.912928,0.891810,0.0,0.108190,0.0,42.1,16.1,NaN,NaN,0.127723,"LINESTRING (2682548.001 1251183.001, 2682500.0..."
2,234507,67262502,36,68252512,15.236510,20.146581,0.873059,0.0,0.126941,0.0,41.1,17.1,NaN,NaN,0.122197,"LINESTRING (2682548.001 1251183.001, 2682500.0..."
3,234507,67262503,36,68252512,4.651834,19.254704,0.856604,0.0,0.143396,0.0,38.1,16.1,NaN,NaN,0.126132,"LINESTRING (2682548.001 1251183.001, 2682500.0..."
4,234507,67262504,36,68252512,159.777359,19.324976,0.846430,0.0,0.153570,0.0,37.1,16.1,NaN,NaN,0.125810,"LINESTRING (2682548.001 1251183.001, 2682500.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441291,12723489,69232494,48,68252512,41.370000,21.204255,0.889250,0.0,0.110750,0.0,38.1,19.1,NaN,NaN,0.117897,"LINESTRING (2682455.001 1251225.001, 2682500.0..."
1441292,12723489,69232496,48,68252512,24.422705,21.204255,0.889250,0.0,0.110750,0.0,38.1,19.1,NaN,NaN,0.117897,"LINESTRING (2682455.001 1251225.001, 2682500.0..."
1441293,12723489,69232497,48,68252512,113.076558,18.993732,0.913921,0.0,0.086079,0.0,39.1,17.1,NaN,NaN,0.127342,"LINESTRING (2682455.001 1251225.001, 2682500.0..."
1441294,12723489,69232498,48,68252512,4.000000,18.938926,0.920047,0.0,0.079953,0.0,40.1,17.1,NaN,NaN,0.127600,"LINESTRING (2682455.001 1251225.001, 2682500.0..."


In [21]:
# for each person, sum the accessibility contributions across all destinations
accessibility = ag.groupby('record').agg({
    'accessibility_contribution': 'sum',
    'accessibility_contribution_cycling': 'sum',
    'accessibility_contribution_foot': 'sum',
    'accessibility_contribution_private_cars': 'sum',
    'accessibility_contribution_transit': 'sum'
})

accessibility.rename(columns={
    'accessibility_contribution': 'accessibility',
    'accessibility_contribution_cycling': 'accessibility_cycling',
    'accessibility_contribution_foot': 'accessibility_foot',
    'accessibility_contribution_private_cars': 'accessibility_private_cars',
    'accessibility_contribution_transit': 'accessibility_transit'
}, inplace=True)

accessibility = accessibility.join(statpop_reduced)
accessibility = gpd.GeoDataFrame(accessibility, crs=2056)
accessibility

,accessibility,accessibility_cycling,accessibility_foot,accessibility_private_cars,accessibility_transit,age,maritalstatus,residencepermit,residentpermit,statent_id,geometry
record,,,,,,,,,,,
234507,1513.566677,1095.878201,305.908312,1898.714759,305.908312,36,2,-2,-2,68252512.0,POINT (2682548.001 1251183.001)
274900,1567.896866,1095.878201,305.908312,1898.714759,305.908312,45,2,-2,-2,68252512.0,POINT (2682502.001 1251169.001)
379693,1385.633649,1095.878201,305.908312,1898.714759,305.908312,10,1,-2,-2,68252512.0,POINT (2682545.001 1251248.001)
417369,1385.633649,1095.878201,305.908312,1898.714759,305.908312,7,1,-2,-2,68252512.0,POINT (2682532.001 1251207.001)
637853,1694.430581,1095.878201,305.908312,1898.714759,305.908312,88,3,-2,-2,68252512.0,POINT (2682545.001 1251248.001)
...,...,...,...,...,...,...,...,...,...,...,...
12328917,1493.490797,1095.878201,305.908312,1898.714759,305.908312,33,1,201,2,68252512.0,POINT (2682545.001 1251248.001)
12387021,1385.633649,1095.878201,305.908312,1898.714759,305.908312,10,1,-2,-2,68252512.0,POINT (2682548.001 1251183.001)
12421141,1532.737040,1095.878201,305.908312,1898.714759,305.908312,39,2,-2,-2,68252512.0,POINT (2682450.001 1251222.001)


In [ ]:
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, 'accessibility.gpkg')
)

In [ ]:
if 0:
    for column in ['transport_mode', 'travel_time']:
        travel_times[column] = travel_times[column].astype(str)
    travel_times.to_file(os.path.join(outputs_path, 'routes.gpkg'))

In [ ]:
tt_computers = {}
tt_computers[MODE_TRANSIT] = r5py.DetailedItinerariesComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.TRANSIT],
    max_time=datetime.timedelta(seconds=30),
    speed_walking=3.6,
    speed_cycling=12.0,
    max_public_transport_rides=1,
    max_bicycle_traffic_stress=3,
    breakdown=False,
)

r5py.RegionalTask

In [ ]:
tt_computers[MODE_TRANSIT].compute_travel_details()

In [ ]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.CAR],
)

In [ ]:
tt_computers[MODE_CYCLING].compute_travel_times()

In [ ]:
origins = geopandas.GeoDataFrame(
    {
        "id": ["origin"],
        "geometry": [G.nodes[335]['geometry']]
    },
    crs="EPSG:2056",
)

destinations = geopandas.GeoDataFrame(
    {
        "id": ["destination_1"],
        "geometry": [G.nodes[173]['geometry']]
    },
    crs="EPSG:2056",
)

In [ ]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.DetailedItinerariesComputer(
    transport_network,
    origins=origins,
    destinations=statent_filtered,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.WALK],
    breakdown=True,
)

routes = tt_computers[MODE_CYCLING].compute_travel_details()
routes['transport_mode'] = routes['transport_mode'].astype(str)
routes['travel_time'] = routes['travel_time'].astype(str)
routes['wait_time'] = routes['wait_time'].astype(str)
snman.io.export_gdf(routes, os.path.join(outputs_path, 'routes.gpkg'))
routes

In [ ]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=statent_filtered,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.CAR],
    breakdown=True,
)

routes = tt_computers[MODE_CYCLING].compute_travel_times()
#routes['transport_mode'] = routes['transport_mode'].astype(str)
#routes['travel_time'] = routes['travel_time'].astype(str)
#routes['wait_time'] = routes['wait_time'].astype(str)
#snman.io.export_gdf(routes, os.path.join(outputs_path, 'routes.gpkg'))
routes

In [ ]:
routes = tt_computers[MODE_CYCLING].compute_travel_details()
(routes['distance'][0] / 1000) / (routes['travel_time'][0].total_seconds() / 3600)